<a href="https://colab.research.google.com/github/NimdaGrogu/Machine_Learning_Projects/blob/dev/Generative%20AI%20Foundations%20/Capstone%20Projects/Project%202%20Medical%20Assistant/Learners_Notebook_Full_Code_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

1. **Critical Care Protocols:** "What is the protocol for managing sepsis in a critical care unit?"

2. **General Surgery:** "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

3. **Dermatology:** "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

4. **Neurology:** "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"


### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
!pip install --upgrade pip -q

In [2]:
# Install required libraries
!pip install -q langchain_community==0.3.27 \
              langchain==0.3.27 \
              chromadb==1.0.15 \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              datasets==4.0.0 \
              evaluate==0.4.5 \
              langchain_openai==0.3.30




In [3]:
# Import core libraries
import os                                                                       # Interact with the operating system (e.g., set environment variables)
import json                                                                     # Read/write JSON data
import requests  # type: ignore                                                 # Make HTTP requests (e.g., API calls); ignore type checker

# Import libraries for working with PDFs and OpenAI
from langchain.document_loaders import PyMuPDFLoader                            # Load and extract text from PDF files
# from langchain_community.document_loaders import PyPDFLoader                    # Load and extract text from PDF files
from openai import OpenAI                                                       # Access OpenAI's models and services

# Import libraries for processing dataframes and text
import tiktoken                                                                 # Tokenizer used for counting and splitting text for models
import pandas as pd                                                             # Load, manipulate, and analyze tabular data

# Import LangChain components for data loading, chunking, embedding, and vector DBs
from langchain.text_splitter import RecursiveCharacterTextSplitter              # Break text into overlapping chunks for processing
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings                        # Create vector embeddings using OpenAI's models  # type: ignore
from langchain.vectorstores import Chroma                                       # Store and search vector embeddings using Chroma DB  # type: ignore

from datasets import Dataset                                                    # Used to structure the input (questions, answers, contexts etc.) in tabular format
from langchain_openai import ChatOpenAI                                         # This is needed since LLM is used in metric computation

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

## Loading OpenAI configuration

In [4]:
import json # Import the json module

# Load the JSON file and extract values
file_name = "Config.json"                                                       # Name of the configuration file
with open(file_name, 'r') as file:                                              # Open the config file in read mode
    config = json.load(file)                                                    # Load the JSON content as a dictionary
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")                                             # Extract the API key from the config
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")                             # Extract the OpenAI base URL from the config

# Store API credentials in environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY                                          # Set API key as environment variable
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE                                 # Set API base URL as environment variable

# Initialize OpenAI client
client = OpenAI()

## Question Answering using LLM

## Question Answering with OpenAI



In [5]:
#Define a decorator functin
import textwrap, re
def clean_output(func=None):
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)

    # Check if the result from the function is a string before attempting to clean
    if isinstance(result, str):
      output = ' '.join(result)
      output = textwrap.fill(output, width=100)

      return output
    return result
  return wrapper

# Define a function to get a response
# @clean_output
def response(user_prompt, max_tokens=300, temperature=0.5, top_p=0.9):   # Complete the code to set default paramenters
    # Create a chat completion using the OpenAI client
    completion = client.chat.completions.create(
        model="gpt-4-turbo",  # Complete the code by specifying the model to be used.
        messages=[
            {"role": "user",
             "content": user_prompt
             }                            # User prompt is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output
        top_p=top_p                                                             # Controls diversity via nucleus sampling
    )
    return completion.choices[0].message.content                                # Return the text content from the model's reply

### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [6]:
q1 = "What is the protocol for managing sepsis in a critical care unit?"

base_prompt_response_1 = response(q1)
print(base_prompt_response_1)


The management of sepsis in a critical care unit is a complex and urgent medical situation, requiring a multidisciplinary approach to improve outcomes and reduce mortality. The protocols for managing sepsis are frequently updated as new evidence and guidelines emerge. As of my last update, the management typically follows guidelines established by leading health organizations such as the Surviving Sepsis Campaign (SSC) and is tailored to the specific needs of the patient. Here’s a general overview of the protocol:

1. **Early Identification and Risk Assessment:**
   - Rapid identification of potential sepsis through clinical evaluation and monitoring of symptoms such as fever, increased heart rate, increased respiratory rate, confusion, and possible hypotension.
   - Use of scoring systems like the Sequential Organ Failure Assessment (SOFA) to assess the severity of sepsis and the risk of organ dysfunction.

2. **Immediate Management:**
   - **Resuscitation:** Immediate initiation of f

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [7]:

q2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

base_prompt_response_2 = response(user_prompt=q2, temperature=0.5, top_p=0.9)
print(base_prompt_response_2)


Appendicitis is an inflammation of the appendix, a small, finger-shaped organ attached to the large intestine. Recognizing the symptoms early is crucial as untreated appendicitis can lead to the appendix bursting, causing a potentially life-threatening infection.

### Common Symptoms of Appendicitis
1. **Abdominal Pain:** The classic symptom is sharp pain that begins around the navel and then moves to the lower right abdomen. The pain typically becomes sharper and more severe over time.
2. **Loss of Appetite**
3. **Nausea and Vomiting**
4. **Fever:** Usually a low-grade fever that may worsen as the condition progresses.
5. **Abdominal Swelling**
6. **Constipation or Diarrhea**
7. **Inability to Pass Gas**

### Diagnosis and Treatment
Diagnosis of appendicitis may involve physical examinations, blood tests to check for signs of infection like a high white blood cell count, urine tests to rule out a urinary tract infection, imaging tests such as an ultrasound, CT scan, or MRI.

### Treat

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [8]:
q3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

base_prompt_response_3 = response(user_prompt=q3, temperature=0.5, top_p=0.9)
print(base_prompt_response_3)

Sudden patchy hair loss, which results in localized bald spots on the scalp, is often diagnosed as alopecia areata. This condition is considered an autoimmune disorder where the immune system mistakenly attacks the hair follicles, leading to hair loss. Here are some effective treatments and potential causes:

### Effective Treatments for Alopecia Areata

1. **Corticosteroids**: These are powerful anti-inflammatory drugs that suppress the immune system. They can be administered through local injections, topical ointment application, or orally.

2. **Minoxidil (Rogaine)**: A topical medication that can be used to promote hair growth in affected areas.

3. **Anthralin (Dritho-Scalp)**: A synthetic, tar-like substance that alters immune function in the affected skin. It is applied for a short duration and then washed off to avoid skin irritation.

4. **Topical Immunotherapy**: This involves inducing an allergic reaction with chemicals like diphencyprone (DPCP) or squaric acid dibutylester 

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [9]:
q4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"

base_prompt_response_4 = response(user_prompt=q4, temperature=0.7, top_p=0.9)
print(base_prompt_response_4)

Treating a brain injury, particularly one that has caused temporary or permanent impairment of brain function, is a complex and multifaceted process that typically involves a team of healthcare professionals. The specific treatment recommended can vary significantly depending on the severity and type of the injury (e.g., traumatic brain injury (TBI), stroke, concussion), the part of the brain affected, and the specific symptoms exhibited by the patient. Below are some general approaches and treatments used for brain injuries:

1. **Immediate Medical Care**: Immediate treatment focuses on stabilizing the patient and can include ensuring an adequate airway, breathing, and circulation; controlling external bleeding; and preventing further injury to the brain and other parts of the body.

2. **Medications**:
   - **Diuretics**: These can be used to reduce pressure inside the brain by increasing urine output to remove excess fluid.
   - **Anti-seizure drugs**: Patients are at risk of seizur

### Storing the responds

In [10]:
responses = pd.DataFrame({
    "Question": [q1, q2, q3, q4],
    "Base Prompt Response": [base_prompt_response_1, base_prompt_response_2, base_prompt_response_3, base_prompt_response_4]
    }
)

## Question Answering using LLM with Prompt Engineering

### Defining the function to Generate a Response From the LLM with Prompt Engineering


In [11]:
# Define a function to get a response from the OpenAI chat model
def response_with_prompt_eng(system_prompt, user_prompt, max_tokens=300, temperature=0.5, top_p=0.9):  # Complete the code to set default paramenters
    # Create a chat completion using the OpenAI client
    completion = client.chat.completions.create(
        model="gpt-4-turbo",                                                        # Complete the code by specifying the model to be used.
        messages=[
            {"role": "system", "content": system_prompt},                       # System prompt sets the assistant's behavior
            {"role": "user", "content": user_prompt}                            # User prompt is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output (0 = deterministic)
        top_p=top_p                                                             # Controls diversity via nucleus sampling
    )
    return completion.choices[0].message.content                                # Return the text content from the model's reply

#### Define the prompts for system and user

In [12]:
system_prompt = ("""
    You are an AI medical assistant specialized in providing comprehensive medical information based on the Merck Manuals.
    Your primary goal is to offer accurate, concise, and helpful information for general understanding only.
    You must strictly adhere to the following guidelines:
    - Provide answers based solely on general medical knowledge.
    - Do NOT provide personalized medical advice, diagnoses, or treatment recommendations for individuals.
    - Always maintain a professional, informative, and neutral tone.
    - Clarify that your responses are for informational purposes only and not a substitute for professional medical consultation.
    - Focus on explaining conditions, symptoms, and general treatment approaches rather than prescribing actions.
    """)



### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [13]:
response_with_prompt_eng_1 = response_with_prompt_eng(system_prompt,q1)
print(response_with_prompt_eng_1)


Sepsis is a life-threatening condition triggered by an infection that leads to an overwhelming immune response, potentially resulting in organ failure and death. Management of sepsis in a critical care unit is highly structured and follows specific protocols to stabilize the patient as quickly as possible. The primary goals are to identify the source of infection, control it, and support any organ functions that might be failing.

Here’s a general overview of the protocol for managing sepsis in a critical care setting:

1. **Early Recognition and Immediate Response:**
   - Rapid identification of potential sepsis through the observation of symptoms such as fever, increased heart rate, increased respiratory rate, confusion, and possible hypotension.
   - Use of the Sequential Organ Failure Assessment (SOFA) score to determine the extent of organ function impairment and the likelihood of sepsis.

2. **Initial Resuscitation:**
   - Immediate initiation of treatment, typically within the f

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [14]:
response_with_prompt_eng_2=response_with_prompt_eng(system_prompt,q2)
print(response_with_prompt_eng_2)

Appendicitis is an inflammation of the appendix, a small, finger-shaped pouch that projects from your colon on the lower right side of your abdomen. The common symptoms of appendicitis include:

1. **Abdominal pain** - This pain typically starts near the belly button and then moves to the lower right side of the abdomen. It usually becomes sharper and more severe as the condition progresses.
2. **Loss of appetite**
3. **Nausea and vomiting**
4. **Fever** - Usually low-grade, but can increase as the condition worsens.
5. **Abdominal swelling**
6. **Constipation or diarrhea**
7. **Inability to pass gas**

Appendicitis is considered a medical emergency, primarily because the appendix can rupture if left untreated, leading to more severe complications.

**Treatment for Appendicitis:**
Appendicitis typically requires surgical intervention. Medication alone is not sufficient to cure appendicitis, as antibiotics can treat infection but do not resolve the underlying inflammation and blockage.


### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [15]:
response_with_prompt_eng_3=response_with_prompt_eng(system_prompt,q3)
print(response_with_prompt_eng_3)


Sudden patchy hair loss, which appears as localized bald spots on the scalp, is often due to a condition called alopecia areata. This is an autoimmune disorder where the immune system attacks the hair follicles, leading to hair loss. The exact cause of alopecia areata is not fully understood, but it is believed to involve a combination of genetic and environmental factors.

### Possible Causes of Alopecia Areata
1. **Genetic predisposition**: A family history of alopecia areata or other autoimmune conditions can increase the risk.
2. **Autoimmune response**: The body's immune system mistakenly attacks healthy hair follicles.
3. **Environmental triggers**: Some evidence suggests that stress, viral infections, or hormonal changes might trigger the onset in susceptible individuals.

### Effective Treatments for Alopecia Areata

#### Medical Treatments
1. **Corticosteroids**: These are powerful anti-inflammatory drugs that can suppress the immune system. They can be administered through lo

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [16]:
response_with_prompt_eng_4=response_with_prompt_eng(system_prompt,q4)
print(response_with_prompt_eng_4)

Treatment for brain injuries, whether temporary or permanent, depends on the severity and location of the injury. Here are general approaches used in the management of brain injuries:

1. **Immediate Medical Care**: Initial treatment focuses on stabilizing the patient. This includes ensuring the patient has a sufficient supply of oxygen, maintaining blood flow, and preventing any further injury to the head or neck.

2. **Medications**:
   - **Diuretics** may be used to reduce swelling in the brain.
   - **Anti-seizure drugs** might be prescribed to prevent seizures, which are a common complication following brain injuries.
   - **Coma-inducing drugs** are sometimes used because a comatose brain requires less oxygen to function, which can be beneficial if blood vessels, damaged by the injury, are unable to supply sufficient oxygen.

3. **Surgery**: In cases where there are blood clots (hematomas), bruising of the brain (contusions), or increased pressure inside the skull, surgery might 

#### Storing the generated output

In [17]:
responses["Response with Prompt Engineering"] = [response_with_prompt_eng_1, response_with_prompt_eng_2, response_with_prompt_eng_3, response_with_prompt_eng_4]
responses

,Question,Base Prompt Response,Response with Prompt Engineering
0,What is the protocol for managing sepsis in a ...,The management of sepsis in a critical care un...,Sepsis is a life-threatening condition trigger...
1,"What are the common symptoms for appendicitis,...",Appendicitis is an inflammation of the appendi...,Appendicitis is an inflammation of the appendi...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, which results in loca...","Sudden patchy hair loss, which appears as loca..."
3,What treatments are recommended for a person w...,"Treating a brain injury, particularly one that...","Treatment for brain injuries, whether temporar..."


## Data Preparation for RAG

### Loading the Data

In [18]:
manual_pdf = "medical_diagnosis_manual.pdf"
pdf_loader = PyMuPDFLoader(manual_pdf)
data = pdf_loader.load()

### Data Overview

In [19]:

for i in range(5):
  print(f"Page Number: {i+1}", end="\n")
  print(data[i].page_content,end="\n")


Page Number: 1
jonathan@nimda.sh
4J6EVYSP7A
meant for personal use by jonathan@nim
shing the contents in part or full is liable
Page Number: 2
jonathan@nimda.sh
4J6EVYSP7A
This file is meant for personal use by jonathan@nimda.sh only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number: 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - Nutrition

### Data Chunking
#### Chunk the PDF into Manageable Text Sections Using a Token-Based Splitter


In [35]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=512,
    chunk_overlap=200,
)

In [36]:
document_chunks = text_splitter.split_documents(data)
#document_chunks = pdf_loader.load_and_split(text_splitter)


In [37]:
len(document_chunks)

10115

### Embedding



### Embeddings from OpenAI

In [38]:
# Initialize the OpenAI Embeddings model with API credentials
embedding_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,                                                     # Your OpenAI API key for authentication
    openai_api_base=OPENAI_API_BASE                                             # The OpenAI API base URL endpoint
)

# Generate embeddings (vector representations) for the first two document chunks
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)      # Embedding for chunk 0
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)      # Embedding for chunk 1

# Check and print the dimension (length) of the embedding vector
print("Dimension of the embedding vector ", len(embedding_1))                   # Typically 1536 or 2048 depending on model

Dimension of the embedding vector  1536


In [39]:
# Verify if both embeddings have the same dimension (should be True)
len(embedding_1) == len(embedding_2)


True

In [25]:

# Return/display the two embedding vectors for further inspection or use
embedding_1, embedding_2

([-0.006342974025756121,
  0.006594567093998194,
  -0.010092061944305897,
  -0.009766054339706898,
  -0.02007072977721691,
  0.02442222274839878,
  -0.01943288929760456,
  -0.020198296755552292,
  -0.02864614687860012,
  -0.0038872864097356796,
  0.029907654970884323,
  0.021204669028520584,
  0.01656969077885151,
  0.0039652446284890175,
  0.011013387702405453,
  0.009121125563979149,
  0.018383994698524475,
  -0.015563319437205791,
  0.012912736274302006,
  -0.0006940949824638665,
  -0.010538550093770027,
  0.008164364844560623,
  -0.014117547310888767,
  0.017505191266536713,
  -0.013380486518144608,
  -0.020042380318045616,
  0.003153769299387932,
  -0.031240032985806465,
  -0.00026266646455042064,
  -0.01919192634522915,
  0.027342116460204124,
  -0.015138092450797558,
  -0.00014894029300194234,
  -0.02167241834104061,
  -0.02673262357711792,
  -0.0016043472569435835,
  0.00439401576295495,
  -0.025017540901899338,
  0.025669556111097336,
  -0.001118879416026175,
  -0.005850419402

### Vector Database



#### Setup Vector Database Directory

In [40]:
out_dir = 'vector_db'    # complete the code to define the name of the vector database

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [34]:
"""
 Building the vector store and saving it to disk for future use
# Initialize an empty Chroma vector store
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory=out_dir
)

# Define a batch size for adding documents to avoid hitting API token limits
batch_size = 300  # Adjusted batch size to avoid the max token error

# Add documents in batches
for i in range(0, len(document_chunks), batch_size):
    batch = document_chunks[i : i + batch_size]
    print(f"Adding batch {i // batch_size + 1}/{(len(document_chunks) -1) // batch_size + 1} with {len(batch)} documents...")
    vector_store.add_documents(documents=batch)

print("Vector store created and saved.")
"""

'\n Building the vector store and saving it to disk for future use\n# Initialize an empty Chroma vector store\nvector_store = Chroma(\n    embedding_function=embedding_model,\n    persist_directory=out_dir\n)\n\n# Define a batch size for adding documents to avoid hitting API token limits\nbatch_size = 300  # Adjusted batch size to avoid the max token error\n\n# Add documents in batches\nfor i in range(0, len(document_chunks), batch_size):\n    batch = document_chunks[i : i + batch_size]\n    print(f"Adding batch {i // batch_size + 1}/{(len(document_chunks) -1) // batch_size + 1} with {len(batch)} documents...")\n    vector_store.add_documents(documents=batch)\n\nprint("Vector store created and saved.")\n'

In [41]:
# Building the vector store and saving it to disk for future use
vectorstore = Chroma.from_documents(
    document_chunks,                                                            # Documents to index
    embedding_model,                                                            # Embedding model for converting text to vectors
    persist_directory=out_dir                                                   # Save vector DB files here
)

BadRequestError: Error code: 400 - {'error': {'message': 'Requested 435318 tokens, max 300000 tokens per request', 'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}

In [ ]:
vectorstore = Chroma(
    persist_directory=out_dir,
    embedding_function=embedding_model
)

In [29]:
vectorstore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7c49eecf0710>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7c49eb232570>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [30]:
vectorstore.similarity_search(q1, k=3)

[Document(metadata={'source': 'medical_diagnosis_manual.pdf', 'page': 2456, 'moddate': '2025-11-07T19:12:15+00:00', 'format': 'PDF 1.7', 'trapped': '', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'author': '', 'total_pages': 4114, 'creationDate': 'D:20120615054440Z', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'file_path': 'medical_diagnosis_manual.pdf', 'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'keywords': '', 'subject': '', 'modDate': 'D:20251107191215Z'}, page_content="Parenteral antibiotics should be given after specimens of blood, body fluids, and wound sites have been\ntaken for Gram stain and culture. Very prompt empiric therapy, started immediately after suspecting\nsepsis, is essential and may be lifesaving. Antibiotic selection requires an educated guess based on the\nsuspected source, clinical setting, knowledge or suspicion of causative organisms and of sensitivity\npatterns common to that spec

### Retriever

### Response Function

## Question Answering using RAG

### Question 1: What is the protocol for managing sepsis in a critical care unit?

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

## Output Evaluation

#### **Evaluation 1: Base Prompt Response Evaluation**

#### **Evaluation 2: Prompt Engineering Response Evaluation**

#### **Evaluation 3: RAG Response Evaluation**

## Actionable Insights and Business Recommendations

-
-
-


<font size=6 color='#4682B4'>Power Ahead</font>
___